In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import time
import json
import glob
import numpy as np
import pandas as pd

In [3]:
import sys
sys.path.append('../')
from fog_dataloader import FogDataloader as fog
from models import multiview_vit as vit
from utils import SparkMET as sm
from utils import engine
import torch
device = "cuda:3" if torch.cuda.is_available() else "cpu"
print(f"Check if there is GPU(s): {torch.cuda.is_available()}")

Check if there is GPU(s): True


In [4]:
data_loader_training, data_loader_validate, data_loader_testing = fog.Fog_DataLoader_npz_cv(batch_size = 1, 
                                                                                            kfold_id  = 0, 
                                                                                            SaveDir = '/data1/fog/SparkMET/EXPs/Nov_CV2_UVT/', 
                                                                                            Exp_name = 'uvt_32_1024_512_8_6_0001_01_20_cv2_0', 
                                                                                            WeightR = False)

#================================ Summary of Dataset ==================#
number of training samples:   8096 | number of training fog cases:   145
number of validation samples: 692 | number of validation fog cases: 11
number of test samples:       2228 | number of test fog cases:       67
#======================================================================#


In [8]:
config = sm.SparkMET_Configs( img_size = 32, in_channel = 388, in_time = 4, embd_size = 1024, mlp_size = 512, 
                                      num_heads = 8, dropout = 0.2, num_layers  = 6, EmbType = 'UVT').return_config()

FogNet = sm.SparkMET(config, 
                           SaveDir = '/data1/fog/SparkMET/EXPs/Nov_CV2_UVT/', 
                           Exp_Name = 'uvt_32_1024_512_8_6_0001_01_20_cv2_0')

model, optimizer, loss_func = FogNet.compile(optmizer = 'adamw', loss = 'BCE', lr = 0.0001, wd = 0.01)

In [7]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(num_params)

39663618


In [6]:
df, uvt_att_out = FogNet.inference(model, data_loader_training
                                                )
print(f"att_maps: {uvt_att_out.shape}")

att_maps: (8096, 388)


In [ ]:
_ = engine._return_variable_scores(df, uvt_att_out, global_case = 'Hit', local_idx  = 139)

In [38]:
_ = engine._return_variable_scores(df, uvt_att_out, global_case = 'Hit', local_idx  = None)

Rank 1: Value = 0.005166946444660425, variable = UGRD_825mb, time_step = 2
Rank 2: Value = 0.004779343493282795, variable = VVEL_750mb, time_step = 1
Rank 3: Value = 0.004711048677563667, variable = VVEL_750mb, time_step = 3
Rank 4: Value = 0.004536717664450407, variable = VVEL_750mb, time_step = 2
Rank 5: Value = 0.003829928580671549, variable = TKE_750mb, time_step = 1
Rank 6: Value = 0.0037680314853787422, variable = Q_775mb, time_step = 2
Rank 7: Value = 0.003691519843414426, variable = VVEL_750mb, time_step = 0
Rank 8: Value = 0.003597346832975745, variable = TMP_950mb, time_step = 2
Rank 9: Value = 0.003521250793710351, variable = RH_900mb, time_step = 2
Rank 10: Value = 0.0034923823550343513, variable = TKE_750mb, time_step = 2


In [39]:
_ = engine._return_variable_scores(df, uvt_att_out, global_case = 'Miss', local_idx  = None)

Rank 1: Value = 0.0045018913224339485, variable = VVEL_750mb, time_step = 2
Rank 2: Value = 0.004397112876176834, variable = VVEL_750mb, time_step = 1
Rank 3: Value = 0.004394848830997944, variable = VVEL_750mb, time_step = 3
Rank 4: Value = 0.004069379065185785, variable = VVEL_750mb, time_step = 0
Rank 5: Value = 0.0035720523446798325, variable = UGRD_825mb, time_step = 1
Rank 6: Value = 0.0034743957221508026, variable = RH_725mb, time_step = 3
Rank 7: Value = 0.0034719991963356733, variable = TKE_750mb, time_step = 1
Rank 8: Value = 0.0034683863632380962, variable = VGRD_725mb, time_step = 1
Rank 9: Value = 0.0034512875135987997, variable = Q_700mb, time_step = 2
Rank 10: Value = 0.003412168473005295, variable = TMP_775mb, time_step = 3


In [40]:
_ = engine._return_variable_scores(df, uvt_att_out, global_case = 'FA', local_idx  = None)

Rank 1: Value = 0.005327929742634296, variable = Q_975mb, time_step = 3
Rank 2: Value = 0.004838284105062485, variable = VVEL_750mb, time_step = 2
Rank 3: Value = 0.004518656991422176, variable = VVEL_750mb, time_step = 3
Rank 4: Value = 0.004142205696552992, variable = VVEL_750mb, time_step = 1
Rank 5: Value = 0.0041240183636546135, variable = Q_700mb, time_step = 2
Rank 6: Value = 0.00400832574814558, variable = TMP_725mb, time_step = 3
Rank 7: Value = 0.00399202574044466, variable = TMP_2maboveground, time_step = 2
Rank 8: Value = 0.00395074812695384, variable = Q_875mb, time_step = 3
Rank 9: Value = 0.003944352734833956, variable = UGRD_700mb, time_step = 3
Rank 10: Value = 0.00393643556162715, variable = RH_2maboveground, time_step = 3


In [37]:
_ = engine._return_variable_scores(df, uvt_att_out, global_case = 'CR', local_idx  = None)

Rank 1: Value = 0.003224237123504281, variable = TKE_900mb, time_step = 3
Rank 2: Value = 0.003103845752775669, variable = VVEL_750mb, time_step = 0
Rank 3: Value = 0.00310280779376626, variable = TKE_900mb, time_step = 0
Rank 4: Value = 0.0030688592232763767, variable = VVEL_750mb, time_step = 2
Rank 5: Value = 0.0030680422205477953, variable = VVEL_750mb, time_step = 3
Rank 6: Value = 0.0030592926777899265, variable = VVEL_975mb, time_step = 3
Rank 7: Value = 0.0030582803301513195, variable = TKE_900mb, time_step = 2
Rank 8: Value = 0.003057109424844384, variable = VGRD_800mb, time_step = 3
Rank 9: Value = 0.003049769438803196, variable = Q_925mb, time_step = 0
Rank 10: Value = 0.0030488467309623957, variable = VVEL_975mb, time_step = 0
